In [1]:
import glob 
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
%pylab inline  
import numpy as np
import pandas as pd
from pathlib import Path
import re
import json
import os
import ast
from scipy.sparse import dok_matrix
from sklearn import svm


Populating the interactive namespace from numpy and matplotlib


In [18]:
def get_API_calls(smali_file,app_name,meta):
    #boolean check to see if a line being checked is within a method
    method_open = False
    method_apis = []

    #regex to search for lines starting with '.method'
    method_open_regex = re.compile('\.method.*')
    #regex to search for lines with '.end method'
    close_method_regex = re.compile('\.end method.*')
    #regex to search for lines starting with 'invoke' 
    invoke_method_regex = re.compile('^invoke.*')
    #regex to get rid of obfuscated APIs ..../a/b/a/..
    obfuscated_regex = re.compile('\/.\/.\/')
    
    #looping through all lines in smali files
    for l in open(smali_file):
        
        #if lines starting with '.method'
        method_regex = method_open_regex.findall(l)
        if len(method_regex) > 0:
            method_open = True
            method_start = method_regex[0].split(' ')[1:]
            method_header = ' '.join(method_start)
            method_name = method_header
        
        #if line starts with '.end method'
        method_end = close_method_regex.findall(l)
        if len(method_end) > 0:
            method_open = False
            meta['code_blocks'].append(method_apis)
            method_apis = []
            
        
        #if we found an API being invoked
        API_call = invoke_method_regex.findall(l.strip())

        if len(API_call) > 0:
            API_call_array = API_call[0].split(' ')
            #type of invoke method
            invoke_method_type = API_call_array[0].split('-')[-1]
            api_call = API_call_array[-1].split('->')
            api_call[1] = re.sub('\(.*\)','()',api_call[1])
            try:
                #package name 
                package = api_call[0]
                #api name
                api_name = '->'.join(api_call)
                
                if len(obfuscated_regex.findall(api_name)) >= 1:
                    continue
 
                method_apis.append(api_name)
                meta['invoke_method'][invoke_method_type].add(api_name)
                meta['packages'][package].add(api_name)
                meta['app_apis'][api_name] += 1

            except e:
                print(e)
    return meta

In [19]:
def get_app_api_calls(app_name):
    
    same_code_block = []
    packages = defaultdict(set)
    invoke_method = defaultdict(set)
    app_apis = defaultdict(int)
    meta = {'invoke_method':invoke_method, 'packages':packages, \
            'code_blocks':same_code_block, 'app_apis': app_apis}
    
    pathlist = Path('../data/APK').glob('**/' + app_name + '/**/*.smali')
    for path in pathlist:
        path_in_str = str(path)
        meta = get_API_calls(path_in_str,app_name,meta)
#         break
    return meta


In [20]:
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError


In [21]:
def save_files(meta,p):
    with open(p + '/invoke_method.json', 'w') as im:
        json.dump(meta['invoke_method'], im, indent=4, default=set_default)
    with open(p + '/packages.json', 'w') as pack:
        json.dump(meta['packages'], pack, indent=4, default=set_default)
    with open(p + '/app_apis.json', 'w') as apis:
        json.dump(meta['app_apis'], apis, indent=4, default=set_default)
    with open(p + '/code_blocks.txt', 'w') as cb:
        for l in meta['code_blocks']:
            cb.write(str(l) + '\n')

In [22]:
def read_files(p,apis):
    with open(p + '/app_apis.json', 'r') as a:
        app_apis = json.load(a)
    #creating apis dict
    for k in app_apis.keys():
        if k not in apis.keys():
            apis[k] = len(apis)

    with open(p + '/invoke_method.json', 'r') as im:
        invoke_method = json.load(im)
    with open(p + '/packages.json', 'r') as pack:
        packages = json.load(pack)
        
    code_blocks = []
    with open(p + '/code_blocks.txt', 'r') as cb:
        for line in cb:
            code_blocks.append(ast.literal_eval(line))
    meta = {'invoke_method':invoke_method, 'packages':packages, \
            'code_blocks':code_blocks, 'app_apis': app_apis}
    return meta, apis
    

In [23]:
def get_category_api_calls(category, apps,apis):
    cat_info = {}
    if not os.path.exists('../data/APK/'+category):
        print('Please scrape apps in ' + category + ' first.')
        return None,0
    pathlist = os.listdir('../data/APK/'+category+'/')
    
    for p in pathlist:
        if not p.startswith('.') and not p.endswith('.apk'):
            if p not in apps:
                apps[p] = len(apps)
            meta_data_path = '../data/apk_data/'
            if not os.path.exists(meta_data_path):
                os.mkdir(meta_data_path)
            cat_data_path = meta_data_path + category +'/'
            if not os.path.exists(cat_data_path):
                os.mkdir(cat_data_path)
                
            app_meta_path = cat_data_path + p
            if not os.path.exists(app_meta_path):
                os.mkdir(app_meta_path)
            
                meta = get_app_api_calls(p)
                save_files(meta,app_meta_path)
            meta,apis = read_files(app_meta_path,apis)
                

            cat_info[p] = meta
    return cat_info, apps, apis
      

In [24]:

def run_pipeline(cats):
    categories = {}
    apps = defaultdict(int)
    apis = {}
    for cat in cats:
        categories[cat], apps, apis = get_category_api_calls(cat,apps,apis)
    return categories, apps,apis

In [9]:
cats = ['beauty','business']
category_data,apps,apis = run_pipeline(cats)

KeyboardInterrupt: 

In [25]:
def create_A_matrix(meta,app_id,mat,apis):
    for api_id in meta.keys():
        mat[(app_id,apis[api_id])] = 1
    return mat

In [26]:
def create_B_matrix(meta,mat,apis):
    for v in meta:
        for i,api_id1 in enumerate(v):
            for j in range(i):
                api_id2 = v[j]
                mat[(apis[api_id2] , apis[api_id1])] = 1
                mat[(apis[api_id1] , apis[api_id2])] = 1
    return mat

In [27]:
def create_P_matrix(meta,mat,apis):
    for k,v in meta.items():
        for api_id1 in v:
            for j in range(i):
                api_id2 = v[j]
                mat[(apis[api_id2] , apis[api_id1])] = 1
                mat[(apis[api_id1], apis[api_id2])] = 1
    return mat

In [28]:
def create_matrices(category_data,app_dict,apis):   
    l_api = len(apis)
    l_app = len(app_dict)
    A = dok_matrix((l_app, l_api),dtype = int) 
    B = dok_matrix((l_api, l_api),dtype = int)
    P = dok_matrix((l_api, l_api),dtype = int)
    
    for apps in category_data.items():
        if not apps[1]: continue
        for app,meta in apps[1].items():
            A = create_A_matrix(meta['app_apis'],app_dict[app],A,apis)
            B = create_B_matrix(meta['code_blocks'],B,apis)
            P = create_P_matrix(meta['packages'],P,apis)
    return A,B,P


In [37]:
def read_malware(cats, path,category_data,apps,apis):
    malware = []
    for cat in cats:
        print(cat)

            
        family_data = {}
        base_path = os.path.join(path, 'apk_data', cat)
        for app in os.listdir(base_path):
            
            if app.startswith("."):
                continue
            app_path = os.path.join(base_path, app)
            if app not in apps:
                apps[app] = len(apps)
                malware.append(apps[app])
            meta,apis = read_files(app_path,apis)
            family_data[app] = meta
        category_data[cat] = family_data
    return category_data,apps,apis, malware

In [53]:
def get_malware(path, num_apps,category_data,apps,apis,categories):
    
    sampled = []
    malware = []
    for family in os.listdir(path):
        if family not in categories:
            continue
        family_data = {}
        base_path = os.path.join('..','data','apk_data',family)
        
        if not os.path.exists(base_path):
            os.mkdir(base_path)



        malware_apps = []
        for variety in os.listdir(os.path.join(path,family)):
            for app in os.listdir(os.path.join(path,family,variety)):
                malware_apps.append(os.path.join(path,family,variety,app))
            
        sampled_apps = sample(malware_apps, min(num_apps, len(malware_apps)))
        for malware_path in sampled_apps:
            app = malware_path.split('/')[-1]
            if app not in apps:
                apps[app] = len(apps)
                malware.append(apps[app])
                
            app_path = os.path.join(base_path, app)
            if not os.path.exists(app_path):
                os.mkdir(app_path)

            meta = get_app_api_calls(app,path,True)
            save_files(meta,app_path)
            meta,apis = read_files(app_path,apis)
            family_data[app] = meta

        category_data[family] = family_data
    return category_data,apps,apis, malware


In [54]:
get_malware("/datasets/dsc180a-wi20-public/Malware/amd_data_smali/",50 ,{},{},{},["Andup","Kyview","Minimob","Youmi"])


TypeError: get_app_api_calls() takes 1 positional argument but 3 were given

In [52]:
from random import sample

In [15]:
A_matrix, B_matrix, P_matrix= create_matrices(category_data,apps,apis)

In [41]:
A_At = (A_matrix* A_matrix.T)
A_At

<13x13 sparse matrix of type '<class 'numpy.int64'>'
	with 169 stored elements in Compressed Sparse Row format>

In [42]:
A_At.shape[0]

13

In [22]:
A_B_At = (A_matrix * B_matrix * A_matrix.T)

In [14]:
def save_matrices(A,B,P):
    path = '../data/matrices/'
    if not os.path.exists(path):
        os.mkdir(path)
    np.save(path + 'A.npy', A)
    np.save(path + 'B.npy', B)
    np.save(path + 'P.npy', P)

In [23]:
A_P_At = (A_matrix * P_matrix * A_matrix.T).toarray()
A_P_At

array([[ 321194,  212319,  272448,  243029,  210623,  323504,  216678,
         119205,  194720,   93059,  249566,   97263,    7469],
       [ 212319,  521888,  251195,  227381,  271134,  218811,  368421,
         112052,  194259,  107025,  227170,  107659,    9307],
       [ 272448,  251195,  736371,  353112,  475204,  282338,  485927,
         129938,  218070,  176495,  358739,  121598,    9623],
       [ 243029,  227381,  353112,  391443,  309582,  247333,  321779,
         128665,  207425,  164218,  354421,  105352,    7690],
       [ 210623,  271134,  475204,  309582,  928021,  221112,  609303,
         117265,  196626,  179416,  306935,  113876,    8990],
       [ 323504,  218811,  282338,  247333,  221112,  421890,  232732,
         121063,  208528,   99693,  254663,  104969,    7929],
       [ 216678,  368421,  485927,  321779,  609303,  232732, 1219380,
         119217,  274463,  188829,  321283,  191586,   10941],
       [ 119205,  112052,  129938,  128665,  117265,  121063, 

In [24]:
A_P_B_Pt_At = A_matrix * P_matrix * B_matrix * P_matrix.T * A_matrix.T
A_P_B_Pt_At

array([[ 482422324,  536405788,  625890271,  515617804,  641216357,
         505593888,  692380454,  252684351,  497070664,  331918337,
         519413791,  287792815,   30382041],
       [ 536405788,  783636184,  733352591,  610085222,  770700644,
         566887993,  882070929,  302174516,  590272648,  397348182,
         610450551,  356295672,   40072552],
       [ 625890271,  733352591, 1160222091,  741632530, 1104338145,
         658034291, 1180076546,  343261428,  674382080,  492784919,
         750493620,  396401263,   43656042],
       [ 515617804,  610085222,  741632530,  665971284,  774786403,
         541790001,  832976771,  287396930,  563259993,  412694483,
         630463924,  327370298,   33871856],
       [ 641216357,  770700644, 1104338145,  774786403, 1360825750,
         677872823, 1260053284,  358063995,  705024407,  517717449,
         773387252,  420764242,   46014839],
       [ 505593888,  566887993,  658034291,  541790001,  677872823,
         564059257,  735431

In [36]:
malware = [1,3,6,9]

In [40]:
A_At

array([[15768,  6968,  9505,  7831,  6050, 15521,  5705,  3833,  6093,
         2055,  8610,  2869,   276],
       [ 6968, 27812,  7891,  7075,  8567,  7277, 12904,  3082,  6354,
         2274,  6952,  3686,   328],
       [ 9505,  7891, 32977, 11834, 15048,  9915, 14677,  3581,  6142,
         4637, 12176,  3606,   353],
       [ 7831,  7075, 11834, 15745,  9752,  7914,  9434,  3831,  5894,
         4423, 13032,  2752,   258],
       [ 6050,  8567, 15048,  9752, 45205,  6572, 22589,  3238,  5365,
         4475,  9483,  3275,   310],
       [15521,  7277,  9915,  7914,  6572, 22196,  6546,  3821,  6867,
         2448,  8724,  3271,   293],
       [ 5705, 12904, 14677,  9434, 22589,  6546, 67688,  2839,  9852,
         4593,  9418,  7553,   351],
       [ 3833,  3082,  3581,  3831,  3238,  3821,  2839,  5996,  5244,
         1330,  3816,  2401,   207],
       [ 6093,  6354,  6142,  5894,  5365,  6867,  9852,  5244, 28381,
         2204,  5795,  6316,   319],
       [ 2055,  2274,  4637,

In [37]:
y = [0] * len(apps)
for app_id in malware:
    y[app_id] = 1

In [38]:
clf = svm.SVC()
clf.fit(A_At, y)

//anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [39]:
clf.score(A_At, y)

1.0

In [ ]:
def get_malware_meta(base):
    
    
get_malware_meta('')

In [ ]:
base = '/datasets/dsc180a-wi20-public/Malware/amd_data_smali/' 
amd_file_path = base + 'amd_data_smali/'


In [14]:
for file in os.listdir('../data'):
    print(file)

report_ap_at.txt
report_apb_pt_at.txt
xml_files
log.txt
report_a_at.txt
apis.json
apps.json
report_ab_at.txt
APK
apk_data


In [24]:
json_fp = '../data/apis.json'


In [25]:
import json
apis = json.load(open(json_fp))

In [28]:
count = 0
for x in apis.keys():
    
    if len(con) > 1:
        count += 1
        print(x)

Lorg/apache/a/b/b/f;->f()Lorg/apache/a/i/d;
Lorg/apache/a/a/e;->c()Lorg/apache/a/a/a;
Lorg/apache/a/a/e;->e()Lorg/apache/a/a/d;
Lorg/apache/a/a/e;->d()Lorg/apache/a/a/h;
Lorg/apache/a/c/m;->l()Lorg/apache/a/c/b/b;
Lorg/apache/a/b/f;->a()Lorg/apache/a/a/h;
Lorg/apache/a/b/a;->a()Lorg/apache/a/a/a;
Lorg/apache/a/d/j;->a()Lorg/apache/a/d/h;
Lorg/apache/a/h/g;->f()Lorg/apache/a/i/d;
Lorg/apache/a/h/i;->a()Lorg/apache/a/k/b;
Lorg/apache/a/e/a/d;->values()[Lorg/apache/a/e/a/d;
Lorg/apache/a/e/a/a;->b()Lorg/apache/a/e/a/a/c;
Lorg/apache/a/e/a/c;->a()Lorg/apache/a/k/a;
Lorg/apache/a/e/a/a;->c()Lorg/apache/a/e/a/b;
Lorg/apache/a/e/a/b;->a()Lorg/apache/a/e/a/f;
Lorg/apache/a/k/g;->a()Lorg/apache/a/k/g;
Lorg/apache/a/a/b;->a()Lorg/apache/a/a/a;
Lorg/apache/a/f/f;->a()Lorg/apache/a/g/f;
Lorg/apache/a/f/f;->b()Lorg/apache/a/g/g;
Lorg/apache/a/f/a;->m()Lorg/apache/a/f/e/b;
Lorg/apache/a/f/a;->l()Lorg/apache/a/f/e/a;
Lorg/apache/a/f/a;->a()Lorg/apache/a/g/c;
Lorg/apache/a/f/a;->a()Lorg/apache/a/g/d;


Lcom/google/a/a/a/c/a/a$a;-><init>()V
Lcom/google/a/b/a/a$a;->k()Lcom/google/a/b/a/a$a;
Lcom/google/a/a/a/c/a/a$a;->e()Lcom/google/a/a/a/c/a/a$a;
Lcom/google/a/a/a/c/a/a$a;->f()Lcom/google/a/a/a/c/a/a$a;
Lcom/google/a/a/a/c/a/a$a;->c()Lcom/google/a/a/a/c/a$a;
Lcom/google/a/b/a/a$a;->i()Lcom/google/a/b/a/a$a;
Lcom/google/a/b/a/a$a;->j()Lcom/google/a/b/a/a$a;
Lcom/google/a/b/a/a$a;->h()Lcom/google/a/b/a/a$a;
Lcom/google/a/a/a/c/a/a$a;->g()Lcom/google/a/a/a/c/a/a$a;
Lcom/google/a/b/a/b;->e()Lcom/google/a/b/a/b;
Lcom/google/a/b/a/a$c$a;->f()Lcom/google/a/b/a/a$c$a;
Lcom/google/a/b/a/b;->d()Lcom/google/a/b/a/b;
Lcom/google/a/b/a/b;->c()Lcom/google/a/b/a/b;
Lcom/google/a/b/a/a$c$a;->b()Lcom/google/a/b/a/a$c$a;
Lcom/google/a/b/a/a$c$a;->a()Lcom/google/a/b/a/a$c$a;
Lcom/google/a/a/a/c/a/b;-><init>()V
Lcom/google/a/a/a/c/a/b;->d()Lcom/google/a/a/a/c/a/a;
Lcom/google/a/a/a/c/a/b;->c()Lcom/google/a/a/a/c/a/b;
Lcom/google/a/b/a/a$b$a;->f()Lcom/google/a/b/a/a$b$a;
Lcom/google/a/b/a/a$b$a;->b()Lcom/

Lcom/a/a/b/a;->b()Lcom/a/a/b/c/a;
Lcom/a/a/b/c;->j()Lcom/a/a/b/a/f;
Lcom/a/a/b/c;->o()Lcom/a/a/b/f/a;
Lcom/a/a/b/c;->p()Lcom/a/a/b/f/a;
Lcom/a/a/b/c;->q()Lcom/a/a/b/c/a;
Lcom/a/a/b/e/a;->c()Lcom/a/a/b/a/l;
Lcom/a/a/b/h;->g()Lcom/a/a/b/d/b;
Lcom/a/a/b/c$a;->a()Lcom/a/a/b/c$a;
Lcom/a/a/b/c$a;->a()Lcom/a/a/b/c;
Lcom/a/a/b/e;->a()Lcom/a/a/b/a/g;
Lcom/a/a/c/a;->a()Lcom/a/a/b/a/g;
Lcom/a/a/b/e$a;->h()Lcom/a/a/b/f/a;
Lcom/a/a/b/e$a;->m()Lcom/a/a/b/a/j;
Lcom/a/a/b/e$a;->n()Lcom/a/a/a/a/b;
Lcom/a/a/b/e$a;->o()Lcom/a/a/a/b/a;
Lcom/a/a/b/e$a;->p()Lcom/a/a/b/c;
Lcom/a/a/b/e$a;->r()Lcom/a/a/b/d/b;
Lcom/a/a/b/e$a;->s()Lcom/a/a/b/b/b;
Lcom/a/a/b/a;->a()Lcom/a/a/a/a/b;
Lcom/a/a/b/e$a;->a()Lcom/a/a/b/e;
Lcom/a/a/b/h;->a()Lcom/a/a/b/e;
Lcom/a/a/b/c$a;->j()Lcom/a/a/b/a/f;
Lcom/a/a/b/c$a;->o()Lcom/a/a/b/f/a;
Lcom/a/a/b/c$a;->p()Lcom/a/a/b/f/a;
Lcom/a/a/b/c$a;->q()Lcom/a/a/b/c/a;
Lcom/a/a/b/a;->a()Lcom/a/a/a/a/b/a;
Lcom/a/a/b/a;->a()Lcom/a/a/a/b/a;
Lcom/a/a/a/b/a/a;-><init>()V
Lcom/a/a/b/a;->a()Lcom/a/a/b/

Lcom/airbnb/lottie/a/b/o;->b()Lcom/airbnb/lottie/a/b/a;
Lcom/airbnb/lottie/a/b/o;->c()Lcom/airbnb/lottie/a/b/a;
Lcom/airbnb/lottie/a/b/o;->a()Lcom/airbnb/lottie/a/b/a;
Lcom/airbnb/lottie/a/a/a$a;->b()Lcom/airbnb/lottie/a/a/r;
Lcom/airbnb/lottie/a/a/r;->f()Lcom/airbnb/lottie/a/b/a;
Lcom/airbnb/lottie/a/a/r;->d()Lcom/airbnb/lottie/a/b/a;
Lcom/airbnb/lottie/a/a/r;->e()Lcom/airbnb/lottie/a/b/a;
Lcom/facebook/ads/internal/i/a/a;->a()Lcom/facebook/ads/internal/i/a/n;
Lcom/facebook/ads/internal/i/a/p;->a()Lcom/facebook/ads/internal/i/a/p;
Lcom/facebook/ads/internal/i/a/a;->b()Lcom/facebook/ads/internal/i/a/n;
Lcom/facebook/ads/internal/i/a/a;->a()Lcom/facebook/ads/internal/i/a/a;
Lcom/facebook/ads/internal/view/d/c/c;->getState()Lcom/facebook/ads/internal/view/d/c/d;
Lcom/facebook/ads/internal/view/d/c/b;->getState()Lcom/facebook/ads/internal/view/d/c/d;
Lcom/facebook/ads/internal/view/d/b/k;->a()Lcom/facebook/ads/internal/view/d/b/l;
Lcom/facebook/ads/internal/view/d/b/g;->a()Lcom/facebook/a

Ld/a/a/a/a/g/a;->a()Z
Ld/a/a/a/a/b/a;-><init>()V
Ld/a/a/a/a/e/d;->a()Ld/a/a/a/a/e/d;
Ld/a/a/a/a/e/d;->e()Ld/a/a/a/a/e/d;
Ld/a/a/a/a/b/i;->d()Z
Ld/a/a/a/a/g/a;->a()Ljava/lang/String;
Ld/a/a/a/a/g/a;->b()Ljava/lang/String;
Ld/a/a/a/a/g/a;->b()Ld/a/a/a/a/e/d;
Ld/a/a/a/a/g/a;->a()Ld/a/a/a/a/e/d;
Ld/a/a/a/a/e/d;->b()I
Ld/a/a/a/a/e/d;->o()Ljava/lang/String;
Ld/a/a/a/a/e/d;->b()Ljava/lang/String;
Ld/a/a/a/a/b/v;->a()I
Ld/a/a/a/a/g/l;-><init>()V
Ld/a/a/a/a/g/l;->a()V
Ld/a/a/a/a/g/l;->a()Ljava/lang/String;
Ld/a/a/a/a/g/l;->a()Z
Ld/a/a/a/a/e/d;->d()Ljava/lang/String;
Ld/a/a/a/a/g/l;->a()Lorg/json/JSONObject;
Ld/a/a/a/a/g/l;->b()Ljava/util/Map;
Ld/a/a/a/a/g/l;->a()Ld/a/a/a/a/e/d;
Ld/a/a/a/a/b/i;->l()I
Ld/a/a/a/a/g/n;-><init>()V
Ld/a/a/a/a/g/q;-><init>()V
Ld/a/a/a/a/g/q$a;->a()Ld/a/a/a/a/g/q;
Ld/a/a/a/a/b/w;-><init>()V
Ld/a/a/a/a/g/k;-><init>()V
Ld/a/a/a/a/g/i;-><init>()V
Ld/a/a/a/a/b/s;->g()Ljava/lang/String;
Ld/a/a/a/a/b/s;->f()Ljava/lang/String;
Ld/a/a/a/a/b/s;->e()Ljava/lang/String;
Ld/a/a/a/a

Lcom/startapp/android/publish/f/a/g/b;->b()Lcom/startapp/android/publish/f/a/a/a;
Lcom/startapp/android/publish/f/a/g/b;->a()Lcom/startapp/android/publish/f/a/d/e;
Lcom/startapp/android/publish/f/a/e/d;->a()Lcom/startapp/android/publish/f/a/a/c;
Lcom/startapp/android/publish/f/a/g/c;->b()Lcom/startapp/android/publish/f/a/d/e;
Lcom/startapp/android/publish/f/a/g/c;->a()Lcom/startapp/android/publish/f/a/a/a;
Lcom/startapp/android/publish/f/a/a/a;->a()Lcom/startapp/android/publish/f/a/a/c;
Lcom/purplebrain/adbuddiz/sdk/e/a/a;->e()Lcom/purplebrain/adbuddiz/sdk/e/a/h;
Lcom/purplebrain/adbuddiz/sdk/e/a/a;->a()Lcom/purplebrain/adbuddiz/sdk/e/a/a;
Lcom/purplebrain/adbuddiz/sdk/e/a/a;->f()Lcom/purplebrain/adbuddiz/sdk/e/a/d;
Lcom/purplebrain/adbuddiz/sdk/e/a/h;->a()[Lcom/purplebrain/adbuddiz/sdk/e/a/h;
Lcom/purplebrain/adbuddiz/sdk/e/a/a/c;->a()Lcom/purplebrain/adbuddiz/sdk/e/a/a/c;
Lcom/purplebrain/adbuddiz/sdk/e/a/i;->f()Lcom/purplebrain/adbuddiz/sdk/e/a/d;
Lcom/purplebrain/adbuddiz/sdk/e/a/h

Lcom/mopub/a/b/o;->a()Lcom/mopub/a/b/o;
Lcom/mopub/mobileads/c/a/b;->j()Lcom/mopub/mobileads/c/a/a;
Lcom/mopub/a/a/a;->a()Lcom/mopub/a/b/j$a;
Lcom/mopub/a/b/j$a;->a()Lcom/mopub/a/b/j$a;
Lcom/mopub/a/b/o;->values()[Lcom/mopub/a/b/o;
Lcom/mopub/a/a/a;->b()Lcom/mopub/a/b/j$a;
Lcom/startapp/android/publish/b/a/e/d;->a()Lcom/startapp/android/publish/b/a/a/c;
Lcom/startapp/android/publish/b/a/g/c;->a()Lcom/startapp/android/publish/b/a/a/a;
Lcom/startapp/android/publish/b/a/a/a;->a()Lcom/startapp/android/publish/b/a/a/c;
Lcom/startapp/android/publish/b/a/g/c;->b()Lcom/startapp/android/publish/b/a/d/e;
Lcom/startapp/android/publish/b/a/g/b;->b()Lcom/startapp/android/publish/b/a/a/a;
Lcom/startapp/android/publish/b/a/g/b;->a()Lcom/startapp/android/publish/b/a/d/e;
Lcom/a/a/a;->a()Lcom/a/a/a;
Lcom/google/c/a/a/a/a/a;->a()V
Landroid/support/constraint/a/a/f;->D()Landroid/support/constraint/a/a/f;
Lcom/bumptech/glide/f/b/a;->a()Lcom/bumptech/glide/f/b/c;
Lcom/bumptech/glide/load/c/f/f;->a()Lcom/bu

Lcom/c/c/a/a;->a()Lcom/c/c/a/a;
Lcom/c/a/j;->a()Lcom/c/a/j;
Lcom/c/a/l;->d()Lcom/c/a/l;
Lcom/c/a/a;->b()Lcom/c/a/a;
Lcom/c/a/f;->b()Lcom/c/a/f;
Lcom/c/a/d;->b()Lcom/c/a/d;
Lcom/c/a/l;->b()Lcom/c/a/l;
Lcom/c/a/j$a;->c()Lcom/c/a/j$a;
Lcom/c/a/f$a;->c()Lcom/c/a/f$a;
Lcom/c/a/f$b;->c()Lcom/c/a/f$b;
Lcom/c/a/g;->a()Lcom/c/a/g;
Lcom/c/a/f;->a()Lcom/c/a/f;
Lc/a/a/a/c;->b()Lc/a/a/a/l;
Lc/a/a/a/a/f/d;-><init>()V
Lc/a/a/a/a/f/d;->a()V
Lc/a/a/a/a/f/b;->a()Landroid/content/SharedPreferences;
Lc/a/a/a/a/f/e;->a()Ljava/lang/Object;
Lc/a/a/a/c;->a()Lc/a/a/a/i;
Lc/a/a/a/a/e/f;->a()Ljavax/net/ssl/SSLSocketFactory;
Lc/a/a/a/a/f/c;-><init>()V
Lc/a/a/a/a/b/q;-><init>()V
Lc/a/a/a/a/f/a;-><init>()V
Lc/a/a/a/a/f/a;->a()Ljava/io/File;
Lc/a/a/a/a/d/l;-><init>()V
Lc/a/a/a/a/b/i;->a()V
Lc/a/a/a/a/d/e$1;-><init>()V
Lc/a/a/a/a/d/e;->a()V
Lc/a/a/a/a/d/a;-><init>()V
Lc/a/a/a/a/b/i;->b()V
Lc/a/a/a/a/g/q$a;->a()Lc/a/a/a/a/g/q;
Lc/a/a/a/a/g/q;->a()Lc/a/a/a/a/g/t;
Lc/a/a/a/a/b/n;->c()Ljava/lang/String;
Lc/a/a/a/a/b/m$1;

Ljzt/max/a/a/c;->b()Ljzt/max/a/b/a;
Lorg/a/b/a/a/a/a;->c()[Ljava/lang/Object;
Lorg/a/a/a/a/b/d;-><init>()V
Lorg/a/a/a/a/a/a/d;-><init>()V
Lorg/a/a/a/a/a/a/e;-><init>()V
Lorg/a/a/a/a/a/a/b;->a()V
Lorg/a/a/a/a/a/a/d;->b()Ljava/lang/String;
Lorg/a/a/a/a/a/a/e;->a()[C
Lorg/a/a/a/a/a/a/e;->b()V
Lorg/a/b/a/a/a/a;->b()V
Lorg/a/a/a/a/a/a/c;->a()V
Lorg/a/a/a/a/a/a/f;-><init>()V
Lorg/a/a/a/a/a/a/c;-><init>()V
Lorg/b/a/ak;->a()Lorg/b/a/c/t;
Lorg/b/a/g;->a()Lorg/b/a/af;
Lorg/b/a/c/f;->a()Lorg/b/a/c/h;
Lorg/b/a/c;->a()Lorg/b/a/b;
Lorg/b/a/c;->b()Lorg/b/a/b;
Lorg/b/a/g;->a()Lorg/b/a/p;
Lorg/b/a/p;->a()Lorg/b/a/c/i;
Lorg/b/a/c/d;->f()Lorg/b/a/c/e;
Lorg/b/a/c/d;->j()Lorg/b/a/c/y;
Lorg/b/a/c/k;->f()Lorg/b/a/c/l;
Lorg/b/a/a/b;->c()Lorg/b/a/r;
Lorg/b/a/j;->w()Lorg/a/a/a/a/a/a/b;
Lorg/b/a/j;->d()Lorg/b/a/k;
Lorg/b/a/c/t;->c()Lorg/b/a/c/v;
Lorg/b/a/c/t;->d()Lorg/b/a/c/u;
Lorg/b/a/af;->e()Lorg/b/a/g;
Lorg/b/a/af;->d()Lorg/b/a/al;
Lorg/b/a/af;->a()Lorg/b/a/al;
Lorg/b/a/g/e;->a()Lorg/b/a/c/i;
Lorg/b/a/g/e;->a

Lcom/b/a/a/g;->a()Lcom/b/a/a/g;
Lcom/b/a/a/d;->c()Lcom/b/a/a/f;
Lcom/a/a/a/a$a;->a()Lcom/a/a/a/a$a;
Lcom/a/a/a/a$a;->b()Lcom/a/a/a/a$a;
Lcom/a/a/a/a$a;->c()Lcom/a/a/a/a$a;
Lcom/a/a/a/a$a;->a()Lcom/a/a/a/a;
Lcom/b/a/a/f;->a()Lcom/b/a/a/f;
Lcom/b/a/a/e$6;->a()Lcom/b/a/a/e;
Lcom/b/a/a/b;->c()Lcom/b/a/a/f;
Lcom/b/a/a/e$6;->a()Lcom/b/a/a/f;
Lcom/b/a/a/c;->c()Lcom/b/a/a/f;
Lcom/a/a/a/b;->a()Lcom/a/a/a/a$b;
Lcom/a/a/a/a;->a()Lcom/a/a/a/a$a;
Lcom/a/a/a/a;->b()Lcom/a/a/a/b;
Lcom/startapp/simple/a/a/b;->a()Lcom/startapp/simple/a/a/b;
Lcom/startapp/simple/a/a/c;->b()Lcom/startapp/simple/a/a/b;
Lcom/startapp/simple/a/a/d;->a()Lcom/startapp/simple/a/a/b;
Lcom/startapp/simple/a/a/a;->a()Lcom/startapp/simple/a/a/a$a;
Lcom/startapp/simple/a/a/b;->a()Lcom/startapp/simple/a/a/g;
Lcom/b/a/a/e;->a()Lcom/b/a/a/e;
Lcom/badlogic/gdx/e/a/h;->b()Lcom/badlogic/gdx/e/a/b;
Lcom/badlogic/gdx/e/a/e;->g()Lcom/badlogic/gdx/e/a/i;
Lcom/badlogic/gdx/e/a/e;->e()Lcom/badlogic/gdx/e/a/h;
Lcom/badlogic/gdx/e/a/f;->j()Lcom/

In [29]:
count

7702

In [71]:
count = 0
apis_2 = {}
api_keys = sorted(apis.keys())
for i,x in enumerate(api_keys):
#     if 'init' in x:
#         count+=1
#     if re.findall('->\w\(\)', x):
#         x = re.sub('->\w\(\)', '->a()', x)
    print(x)

LA;-><init>()V
LAA;-><init>()V
LAA;->a()LDB;
LAA;->a()LVia;
LAB;-><init>()V
LAC;-><init>()V
LAC;->a()V
LAC;->b()V
LAC;->c()V
LAD;->a()V
LAE;-><init>()V
LAE;->A()J
LAE;->B()I
LAE;->C()J
LAE;->D()Ljava/lang/String;
LAE;->E()Ljava/lang/String;
LAE;->a()J
LAE;->a()Ljava/util/List;
LAE;->a()LxF;
LAE;->a()V
LAE;->b()Ljava/lang/String;
LAE;->b()Ljava/util/List;
LAE;->c()V
LAE;->f()Ljava/util/Map;
LAE;->v()V
LAE;->w()V
LAE;->x()Landroid/database/sqlite/SQLiteDatabase;
LAE;->y()Z
LAE;->z()V
LAF;-><init>()V
LAF;->a()V
LAF;->close()V
LAF;->write()V
LAG;-><init>()V
LAG;->a()V
LAG;->a()Z
LAG;->b()V
LAH;-><init>()V
LAI;->a()LyI;
LAJ;-><init>()V
LAJ;->a()LAJ;
LAJ;->a()LCJ;
LAJ;->toString()Ljava/lang/String;
LAK;-><init>()V
LAL;-><init>()V
LAM;-><init>()V
LAN;->a()V
LAO;-><init>()V
LAP;-><init>()V
LAQ;->a()Ljava/lang/Object;
LAR;-><init>()V
LAR;->a()Ljava/lang/String;
LAR;->b()Ljava/lang/String;
LAR;->c()Ljava/lang/String;
LAR;->d()Ljava/lang/String;
LAR;->e()Z
LAR;->f()Ljava/lang/String;
LAR;->g()Lja

LIka;->a()LDB;
LIl;-><init>()V
LIl;->a()V
LIla;-><init>()V
LIm;-><init>()V
LIm;->a()LRC;
LIm;->a()V
LIm;->a()Z
LIm;->b()V
LIm;->gc()V
LIma;-><init>()V
LIn;-><init>()V
LIn;->a()LIn;
LIn;->a()V
LIn;->b()LIn;
LIn;->c()LIn;
LIn;->d()LIn;
LIn;->e()Ljava/util/Map;
LIna;-><init>()V
LIp;-><init>()V
LIpa;-><init>()V
LIq;-><init>()V
LIq;->a()Landroid/support/v4/util/ArrayMap;
LIq;->a()Lyq;
LIqa;-><init>()V
LIr;-><init>()V
LIra;-><init>()V
LIs;-><init>()V
LIs;->a()V
LIsa;-><init>()V
LIsa;->a()Ljavax/microedition/khronos/egl/EGLConfig;
LIsa;->a()V
LIsa;->b()V
LIsa;->c()V
LIsa;->d()Landroid/graphics/Bitmap;
LIt;-><init>()V
LIt;->a()Ljava/lang/String;
LIta;->iterator()Ljava/util/Iterator;
LIu;-><init>()V
LIu;->a()I
LIu;->a()LIu;
LIu;->a()Landroid/app/PendingIntent;
LIu;->a()Landroid/content/Intent;
LIu;->a()Ljava/lang/String;
LIu;->b()V
LIu;->b()Z
LIu;->c()I
LIu;->c()Landroid/content/Intent;
LIv;-><init>()V
LIw;-><init>()V
LIx;-><init>()V
LIx;->a()V
LIy;-><init>()V
LIy;->a()LGy;
LIz;-><init>()V
LIz;

LSU;->j()LSU;
LSU;->k()LSU;
LSV;-><init>()V
LSV;->a()LTV;
LSV;->a()Lhma;
LSV;->b()LKq;
LSV;->b()LTV;
LSW;-><init>()V
LSW;->a()Ljava/lang/String;
LSW;->a()V
LSX;-><init>()V
LSY;-><init>()V
LSY;->c()LYY;
LSZ;->a()Z
LSZ;->b()LUZ;
LSZ;->c()I
LSa;-><init>()V
LSaa;-><init>()V
LSaa;->a()I
LSaa;->a()V
LSaa;->b()LTaa;
LSaa;->b()Z
LSaa;->c()I
LSaa;->c()LTaa;
LSaa;->clone()Ljava/lang/Object;
LSaa;->d()I
LSaa;->equals()Z
LSaa;->hashCode()I
LSb;-><init>()V
LSb;->a()Landroid/os/ParcelFileDescriptor;
LSb;->a()V
LSba;-><init>()V
LSc;-><init>()V
LSc;->a()Lad;
LSc;->a()V
LSc;->b()Lad;
LSca;-><init>()V
LSd;->a()LRd;
LSda;-><init>()V
LSda;->a()I
LSda;->a()LSda;
LSda;->b()I
LSda;->c()I
LSda;->j()LSda;
LSda;->k()Lnj;
LSe;-><init>()V
LSea;-><init>()V
LSevenZip/CRC;-><init>()V
LSevenZip/CRC;->GetDigest()I
LSevenZip/CRC;->Init()V
LSevenZip/CRC;->Update()V
LSevenZip/CRC;->UpdateByte()V
LSevenZip/Compression/LZ/BinTree;-><init>()V
LSevenZip/Compression/LZ/BinTree;->Create()Z
LSevenZip/Compression/LZ/BinTree;->Ge

LaM;->f()[LaM;
LaN;-><init>()V
LaN;->a()I
LaN;->a()LRN;
LaN;->a()Ljava/util/List;
LaN;->a()V
LaN;->a()Z
LaN;->b()LRN;
LaN;->c()Ljava/util/concurrent/ExecutorService;
LaN;->d()Ljava/lang/String;
LaN;->e()Ljava/lang/String;
LaN;->f()Ljava/lang/String;
LaN;->g()LjN;
LaN;->h()LER;
LaN;->i()Ljava/util/List;
LaN;->j()LVla;
LaN;->k()Landroid/content/Context;
LaN;->l()Z
LaN;->m()LXN;
LaN;->n()Lnu;
LaO;-><init>()V
LaO;->a()Landroid/content/Context;
LaO;->a()V
LaO;->a()Z
LaO;->b()V
LaO;->b()Z
LaO;->c()I
LaO;->c()Z
LaO;->d()Ljava/lang/Object;
LaO;->d()Z
LaO;->e()LaO;
LaO;->e()LyN;
LaO;->f()LyN;
LaP;-><init>()V
LaQ;-><init>()V
LaR;-><init>()V
LaS;-><init>()V
LaS;->a()V
LaT;->a()[B
LaU;-><init>()V
LaU;->a()LaU;
LaU;->c()I
LaU;->values()[LaU;
LaV;-><init>()V
LaW;-><init>()V
LaX;-><init>()V
LaY;-><init>()V
LaZ;-><init>()V
LaZ;->e()I
LaZ;->size()I
Laa;-><init>()V
Laa;->a()I
Laa;->a()Laa;
Laa;->a()Landroid/widget/Button;
Laa;->a()Lcom/weibo/sdk/android/Oauth2AccessToken;
Laa;->a()Ljava/lang/String;
Laa

Labi22_0_0/com/facebook/react/modules/i18nmanager/I18nUtil;->isRTLAllowed()Z
Labi22_0_0/com/facebook/react/modules/i18nmanager/I18nUtil;->isRTLForced()Z
Labi22_0_0/com/facebook/react/modules/i18nmanager/I18nUtil;->setPref()V
Labi22_0_0/com/facebook/react/modules/image/ImageLoaderModule$1;-><init>()V
Labi22_0_0/com/facebook/react/modules/image/ImageLoaderModule$2;-><init>()V
Labi22_0_0/com/facebook/react/modules/image/ImageLoaderModule$3;-><init>()V
Labi22_0_0/com/facebook/react/modules/image/ImageLoaderModule$3;->doInBackgroundGuarded()V
Labi22_0_0/com/facebook/react/modules/image/ImageLoaderModule$3;->executeOnExecutor()Landroid/os/AsyncTask;
Labi22_0_0/com/facebook/react/modules/image/ImageLoaderModule;-><init>()V
Labi22_0_0/com/facebook/react/modules/image/ImageLoaderModule;->access$000()Lcom/facebook/datasource/DataSource;
Labi22_0_0/com/facebook/react/modules/image/ImageLoaderModule;->getReactApplicationContext()Labi22_0_0/com/facebook/react/bridge/ReactApplicationContext;
Labi22_

...

Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$2;-><init>()V
Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$2;->doInBackgroundGuarded()V
Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$2;->execute()Landroid/os/AsyncTask;
Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$3;-><init>()V
Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$3;->doInBackgroundGuarded()V
Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$3;->execute()Landroid/os/AsyncTask;
Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$4;-><init>()V
Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$4;->doInBackgroundGuarded()V
Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$4;->execute()Landroid/os/AsyncTask;
Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$5;-><init>()V
Labi23_0_0/com/facebook/react/modules/storage/AsyncStorageModule$5;->doInBackgroundGuarded()V
Labi2

Labi23_0_0/host/exp/exponent/modules/api/components/gesturehandler/react/RNGestureHandlerButtonViewManager;->createViewInstance()Labi23_0_0/host/exp/exponent/modules/api/components/gesturehandler/react/RNGestureHandlerButtonViewManager$ButtonViewGroup;
Labi23_0_0/host/exp/exponent/modules/api/components/gesturehandler/react/RNGestureHandlerButtonViewManager;->onAfterUpdateTransaction()V
Labi23_0_0/host/exp/exponent/modules/api/components/gesturehandler/react/RNGestureHandlerEnabledRootView$1;-><init>()V
Labi23_0_0/host/exp/exponent/modules/api/components/gesturehandler/react/RNGestureHandlerEnabledRootView$RootViewGestureHandler;-><init>()V
Labi23_0_0/host/exp/exponent/modules/api/components/gesturehandler/react/RNGestureHandlerEnabledRootView$RootViewGestureHandler;->begin()V
Labi23_0_0/host/exp/exponent/modules/api/components/gesturehandler/react/RNGestureHandlerEnabledRootView$RootViewGestureHandler;->end()V
Labi23_0_0/host/exp/exponent/modules/api/components/gesturehandler/react/RN

Labi24_0_0/com/facebook/react/modules/datepicker/DatePickerDialogModule;->access$100()Labi24_0_0/com/facebook/react/bridge/ReactApplicationContext;
Labi24_0_0/com/facebook/react/modules/datepicker/DatePickerDialogModule;->createFragmentArguments()Landroid/os/Bundle;
Labi24_0_0/com/facebook/react/modules/datepicker/DatePickerDialogModule;->getCurrentActivity()Landroid/app/Activity;
Labi24_0_0/com/facebook/react/modules/datepicker/DatePickerDialogModule;->getReactApplicationContext()Labi24_0_0/com/facebook/react/bridge/ReactApplicationContext;
Labi24_0_0/com/facebook/react/modules/datepicker/DatePickerMode;-><init>()V
Labi24_0_0/com/facebook/react/modules/datepicker/DatePickerMode;->ordinal()I
Labi24_0_0/com/facebook/react/modules/datepicker/DatePickerMode;->valueOf()Labi24_0_0/com/facebook/react/modules/datepicker/DatePickerMode;
Labi24_0_0/com/facebook/react/modules/datepicker/DatePickerMode;->values()[Labi24_0_0/com/facebook/react/modules/datepicker/DatePickerMode;
Labi24_0_0/com/face

Labi24_0_0/host/exp/exponent/modules/api/av/player/PlayerData$LoadCompletionListener;->onLoadSuccess()V
Labi24_0_0/host/exp/exponent/modules/api/av/player/PlayerData$SetStatusCompletionListener;->onSetStatusComplete()V
Labi24_0_0/host/exp/exponent/modules/api/av/player/PlayerData$SetStatusCompletionListener;->onSetStatusError()V
Labi24_0_0/host/exp/exponent/modules/api/av/player/PlayerData$StatusUpdateListener;->onStatusUpdate()V
Labi24_0_0/host/exp/exponent/modules/api/av/player/PlayerData$VideoSizeUpdateListener;->onVideoSizeUpdate()V
Labi24_0_0/host/exp/exponent/modules/api/av/player/PlayerData;-><init>()V
Labi24_0_0/host/exp/exponent/modules/api/av/player/PlayerData;->access$000()V
Labi24_0_0/host/exp/exponent/modules/api/av/player/PlayerData;->applyNewStatus()V
Labi24_0_0/host/exp/exponent/modules/api/av/player/PlayerData;->beginUpdatingProgressIfNecessary()V
Labi24_0_0/host/exp/exponent/modules/api/av/player/PlayerData;->callStatusUpdateListener()V
Labi24_0_0/host/exp/exponent/mo

KeyboardInterrupt: 

In [66]:
sum(list(apis_2.values())) - len(apis_2.keys())

242499

In [1]:
x = [[1,1,1,1],[1,0,0,1],[0,1,1,0],[0,0,0,0]]
y = [0,1,2,0]

In [7]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import multilabel_confusion_matrix
import numpy as np

In [14]:
clf = svm.LinearSVC(C=0.1, multi_class = 'ovr', dual = False)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
target_names = ['Benign'] + ['Andup', 'Minibox']
report = classification_report(y_test,preds, target_names=target_names)
report_2 = multilabel_confusion_matrix(y_test, preds)

with open('report.txt','w') as f:
    f.write(report)
    for i,m in enumerate(report_2):
        f.write(target_names[i])
        f.write('\n')
        np.savetxt(f, m.round(2), delimiter=",")

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
for m in report_2:
    print(m)
    print()

[[0. 1.]
 [1. 0.]]

[[1. 0.]
 [1. 0.]]

[[1. 1.]
 [0. 0.]]



In [23]:
def create_baseline_df(meta_cat):
    api_count = defaultdict(int)
    for app,v in meta_cat.items():
        for api, count in v['app_apis'].items():
            api_count[api] += count
    return api_count

In [26]:
cat_apis = {}
cat_apis['business'] = create_baseline_df(category_data['business'])
cat_apis['beauty'] = create_baseline_df(category_data['beauty'])

In [37]:
merged = pd.DataFrame(cat_apis).dropna().sort_values('beauty')
merged['abs_diff'] = abs(merged.beauty - merged.business)
merged['smaller']=merged.apply(lambda x: x['beauty'] if x['beauty'] < x['business'] else x['business'], axis = 1) 
merged['perc_diff'] = merged['abs_diff'] / merged['smaller']
(merged.sort_values('perc_diff', ascending= False).head(10)[['beauty','business','perc_diff']])
    

,beauty,business,perc_diff
Ljava/lang/Short;->valueOf()Ljava/lang/Short;,333.0,2.0,165.5
Ljava/util/ResourceBundle;->getString()Ljava/lang/String;,162.0,1.0,161.0
Landroid/util/SparseIntArray;->put()V,123.0,1.0,122.0
Lcom/google/android/gms/common/api/BaseImplementation$b;->b()V,114.0,1.0,113.0
Landroid/widget/Button;->setTextColor()V,1.0,106.0,105.0
Ljava/util/concurrent/atomic/AtomicInteger;->decrementAndGet()I,186.0,2.0,92.0
Ljava/util/ResourceBundle;->getBundle()Ljava/util/ResourceBundle;,81.0,1.0,80.0
Ljava/util/concurrent/atomic/AtomicReference;->getAndSet()Ljava/lang/Object;,156.0,2.0,77.0
[Z->clone()Ljava/lang/Object;,73.0,1.0,72.0
Lcom/google/android/gms/common/internal/safeparcel/zzb;->zzl()F,71.0,1.0,70.0


EDA for A2

In [5]:
cat_apis_by_app = defaultdict(lambda: defaultdict(list))
cat_apis = defaultdict(list)
invoke_type_by_app = defaultdict(lambda: defaultdict(list))
for cat,v in meta.items():
    for app, w in v.items():  
        for smali_files in w:
            for smali_file, calls in smali_files.items():
                for call in calls:
                    cat_apis_by_app[cat][app].append(call['method_name'])
                    cat_apis[cat].append(call['method_name'])
                    invoke_type_by_app[cat][app].append(call['invoke_method'])


In [14]:
s1= 0
for app, calls in cat_apis_by_app['business'].items():
    s1 += (len(calls))
s1/10

114722.7

In [15]:
s2 = 0
for app, calls in cat_apis_by_app['beauty'].items():
    s2 += (len(calls))
s2/10

140575.4

In [95]:
def get_imp_apis(cat_apis):
    most_business = pd.DataFrame(cat_apis['business'], columns = ['Business'])['Business'].value_counts().to_frame()
    most_beauty = pd.DataFrame(cat_apis['beauty'], columns = ['Beauty'])['Beauty'].value_counts().to_frame()
    merged = most_beauty.merge(most_business, left_index = True, right_index = True)
    merged['abs_diff'] = abs(merged.Beauty - merged.Business)
    merged['smaller']=merged.apply(lambda x: x['Beauty'] if x['Beauty'] < x['Business'] else x['Business'], axis = 1) 
    merged['perc_diff'] = merged['abs_diff'] / merged['smaller']
    imp_apis = (merged.sort_values('perc_diff', ascending= False).head(10)[['Beauty','Business','perc_diff']])
    display(imp_apis)
    return imp_apis

get_imp_apis(cat_apis)


,Beauty,Business,perc_diff
public final getView(ILandroid/view/View;Landroid/view/ViewGroup;)Landroid/view/View;,1221,2,609.500000
private f()V,355,1,354.000000
a()V,667,2,332.500000
public getSerializedSize()I,2,535,266.500000
private ac()V,686,3,227.666667
private c()V,836,4,208.000000
public b()Z,200,1,199.000000
public static a(Ljava/lang/String;)Ljava/lang/String;,490,3,162.333333
public final a()Z,294,2,146.000000
public final onPrepared(Landroid/media/MediaPlayer;)V,264,2,131.000000


,Beauty,Business,perc_diff
public final getView(ILandroid/view/View;Landroid/view/ViewGroup;)Landroid/view/View;,1221,2,609.500000
private f()V,355,1,354.000000
a()V,667,2,332.500000
public getSerializedSize()I,2,535,266.500000
private ac()V,686,3,227.666667
private c()V,836,4,208.000000
public b()Z,200,1,199.000000
public static a(Ljava/lang/String;)Ljava/lang/String;,490,3,162.333333
public final a()Z,294,2,146.000000
public final onPrepared(Landroid/media/MediaPlayer;)V,264,2,131.000000


In [93]:
cat_apis

defaultdict(list,
            {'business': ['constructor <init>(Lorg/apache/cordova/NativeToJsMessageQueue$OnlineEventsBridgeMode;)V',
              'public run()V',
              'public run()V',
              'public run()V',
              'public run()V',
              'protected constructor <init>(Lorg/apache/cordova/CordovaWebViewImpl;)V',
              'public clearLoadTimeoutTimer()V',
              'public onDispatchKeyEvent(Landroid/view/KeyEvent;)Ljava/lang/Boolean;',
              'public onDispatchKeyEvent(Landroid/view/KeyEvent;)Ljava/lang/Boolean;',
              'public onDispatchKeyEvent(Landroid/view/KeyEvent;)Ljava/lang/Boolean;',
              'public onDispatchKeyEvent(Landroid/view/KeyEvent;)Ljava/lang/Boolean;',
              'public onDispatchKeyEvent(Landroid/view/KeyEvent;)Ljava/lang/Boolean;',
              'public onDispatchKeyEvent(Landroid/view/KeyEvent;)Ljava/lang/Boolean;',
              'public onDispatchKeyEvent(Landroid/view/KeyEvent;)Ljava/lang/Boolea

In [68]:
most_business = pd.DataFrame(cat_apis['business'], columns = ['Business'])['Business'].value_counts().to_frame()
most_beauty = pd.DataFrame(cat_apis['beauty'], columns = ['Beauty'])['Beauty'].value_counts().to_frame()
merged = most_beauty.merge(most_business, left_index = True, right_index = True)
merged['abs_diff'] = abs(merged.Beauty - merged.Business)
merged['smaller']=merged.apply(lambda x: x['Beauty'] if x['Beauty'] < x['Business'] else x['Business'], axis = 1) 
merged['perc_diff'] = merged['abs_diff'] / merged['smaller']
imp_apis = list(merged.sort_values('perc_diff', ascending= False).head(10).index)

In [96]:
def get_features(app):
    feats = [0 for ap in imp_apis]
    for call in app:
        if call in imp_apis:
            feats[imp_apis.index(call)] = 1
    return feats
            
feats = []
label = []
for cat,v in cat_apis_by_app.items():
    for app_name, calls in v.items():
        feats.append(get_features(calls))
        if cat == 'beauty':
            label.append(0)
        else:
            label.append(1)


In [97]:
df = pd.DataFrame(feats, columns = imp_apis)
# df['label'] = label
df

,public final getView(ILandroid/view/View;Landroid/view/ViewGroup;)Landroid/view/View;,private f()V,a()V,public getSerializedSize()I,private ac()V,private c()V,public b()Z,public static a(Ljava/lang/String;)Ljava/lang/String;,public final a()Z,public final onPrepared(Landroid/media/MediaPlayer;)V
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,1,0,0
2,1,0,1,0,0,0,1,1,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0
6,0,0,0,0,1,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,1,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0


In [102]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


clf = RandomForestClassifier()
# clf = svm.SVC()
# clf = GradientBoostingClassifier()

In [103]:

clf.fit(feats, label)
# preds = clf.predict(df)
# mean(preds == label)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [90]:
from sklearn.metrics import classification_report
print(classification_report(label,preds))

              precision    recall  f1-score   support

           0       1.00      0.60      0.75        10
           1       0.71      1.00      0.83        10

    accuracy                           0.80        20
   macro avg       0.86      0.80      0.79        20
weighted avg       0.86      0.80      0.79        20



In [352]:
for k,v in invoke_method.items():
    for invoke_method1, api1 in enumerate(v):
        for invoke_method2, api2 in enumerate(v):

            api_key_id1 = apis[api1]
            api_key_id2 = apis[api2]

            I_matrix[api_key_id1][api_key_id2] = 1



In [311]:
I_matrix = np.array([[0 for x in apis] for y in apis])
I_matrix.shape

(181, 181)

In [339]:
A_matrix = np.array([[0 for x in apis] for y in meta])
labs = [0 * len(A_At)]

In [ ]:
for cat,v in meta.items():
    for i,apps in enumerate(v.items()):
        for files in apps[1]:
            for calls in files.items():
                for call in calls[1]:
                    A_matrix[i][call['API_name']] = 1


In [341]:
len(apis)

181

In [344]:
A_At = np.matmul(A_matrix,A_matrix.T)

In [345]:
num_of_apps = len(A_At)
num_of_apps

1

In [346]:
len(A_At)

1

N

In [347]:
num_of_apis = len(apis)
num_of_apis

181

In [230]:
number_of_apis

0

In [239]:
number_of_apps

8

In [261]:
num_of_apis_per_app = [amat.sum() for amat in A_matrix]
num_of_apis_per_app

[7981, 6298, 11178, 3630, 15506, 11276, 14844, 80]

In [262]:
avg_num_apis_per_app = mean(num_of_apis_per_app)
avg_num_apis_per_app

8849.125

In [244]:
number_of_apis_per_app

[7981, 6298, 11178, 3630, 15506, 11276, 14844, 80]

In [263]:
num_of_apps_api_isin = [at.sum() for at in A_matrix.T]
num_of_apps_api_isin

[8,
 4,
 5,
 4,
 5,
 4,
 5,
 7,
 7,
 5,
 6,
 5,
 6,
 5,
 6,
 4,
 4,
 4,
 5,
 8,
 8,
 4,
 4,
 8,
 7,
 4,
 4,
 4,
 4,
 5,
 7,
 4,
 5,
 6,
 4,
 6,
 6,
 7,
 7,
 5,
 7,
 4,
 8,
 5,
 5,
 5,
 5,
 5,
 7,
 4,
 4,
 4,
 7,
 7,
 7,
 4,
 7,
 7,
 7,
 7,
 6,
 6,
 5,
 7,
 4,
 7,
 4,
 4,
 5,
 5,
 5,
 4,
 5,
 4,
 5,
 4,
 4,
 4,
 5,
 4,
 4,
 4,
 7,
 6,
 5,
 7,
 4,
 4,
 4,
 7,
 7,
 4,
 5,
 2,
 2,
 2,
 6,
 3,
 4,
 3,
 5,
 7,
 5,
 7,
 4,
 5,
 7,
 4,
 5,
 7,
 4,
 5,
 2,
 5,
 2,
 2,
 3,
 2,
 5,
 5,
 6,
 7,
 6,
 7,
 5,
 7,
 7,
 4,
 7,
 4,
 5,
 7,
 7,
 7,
 5,
 5,
 4,
 7,
 5,
 7,
 7,
 7,
 2,
 2,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 4,
 5,
 2,
 4,
 6,
 5,
 6,
 4,
 7,
 4,
 5,
 7,
 4,
 4,
 4,
 5,
 5,
 4,
 4,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 4,
 6,
 2,
 2,
 4,
 7,
 8,
 7,
 6,
 7,
 7,
 2,
 3,
 5,
 7,
 5,
 2,
 5,
 4,
 2,
 2,
 5,
 2,
 7,
 7,
 7,
 2,
 7,
 7,
 7,
 4,
 7,
 7,
 4,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 4,
 5,
 4,
 4,
 4,
 7,
 4,
 4,
 4,
 4,
 7,
 4,
 5,
 4,
 4,
 8,
 8,
 8,


In [266]:
avg_number_of_apps_an_api_isin = mean(num_of_apps_api_isin)
avg_number_of_apps_an_api_isin

1.759531739324949

In [252]:
number_of_apps_api_isin

[8,
 4,
 5,
 4,
 5,
 4,
 5,
 7,
 7,
 5,
 6,
 5,
 6,
 5,
 6,
 4,
 4,
 4,
 5,
 8,
 8,
 4,
 4,
 8,
 7,
 4,
 4,
 4,
 4,
 5,
 7,
 4,
 5,
 6,
 4,
 6,
 6,
 7,
 7,
 5,
 7,
 4,
 8,
 5,
 5,
 5,
 5,
 5,
 7,
 4,
 4,
 4,
 7,
 7,
 7,
 4,
 7,
 7,
 7,
 7,
 6,
 6,
 5,
 7,
 4,
 7,
 4,
 4,
 5,
 5,
 5,
 4,
 5,
 4,
 5,
 4,
 4,
 4,
 5,
 4,
 4,
 4,
 7,
 6,
 5,
 7,
 4,
 4,
 4,
 7,
 7,
 4,
 5,
 2,
 2,
 2,
 6,
 3,
 4,
 3,
 5,
 7,
 5,
 7,
 4,
 5,
 7,
 4,
 5,
 7,
 4,
 5,
 2,
 5,
 2,
 2,
 3,
 2,
 5,
 5,
 6,
 7,
 6,
 7,
 5,
 7,
 7,
 4,
 7,
 4,
 5,
 7,
 7,
 7,
 5,
 5,
 4,
 7,
 5,
 7,
 7,
 7,
 2,
 2,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 4,
 5,
 2,
 4,
 6,
 5,
 6,
 4,
 7,
 4,
 5,
 7,
 4,
 4,
 4,
 5,
 5,
 4,
 4,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 4,
 6,
 2,
 2,
 4,
 7,
 8,
 7,
 6,
 7,
 7,
 2,
 3,
 5,
 7,
 5,
 2,
 5,
 4,
 2,
 2,
 5,
 2,
 7,
 7,
 7,
 2,
 7,
 7,
 7,
 4,
 7,
 7,
 4,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 4,
 5,
 4,
 4,
 4,
 7,
 4,
 4,
 4,
 4,
 7,
 4,
 5,
 4,
 4,
 8,
 8,
 8,
